## Import Libraries
___

In [307]:
from urllib.request import urlopen
import pandas as pd
from shutil import unpack_archive
import requests, zipfile, io
import urllib.request
from bs4 import BeautifulSoup
import csv
import os

## Download Datasets
___ 



#### Create data directory

In [308]:
os.mkdir('./data')

#### Text correction data set
url1 = https://www.comp.nus.edu.sg/~nlp/sw/10gec_annotations.zip

In [14]:
url = 'http://www.comp.nus.edu.sg/~nlp/sw/sm_norm_mt.tar.gz'
urllib.request.urlretrieve(url, './data/sm_norm_mt.tar.gz')
unpack_archive('./data/sm_norm_mt.tar.gz', './data/sm_norm_mt')

#### Grammar Check dataset

In [111]:
url = 'https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz'
urllib.request.urlretrieve(url, './data/conll14st-test-data.tar.gz')
unpack_archive('./data/conll14st-test-data.tar.gz', './data/conll14st-test')

#### Social media text normalization data
url2 = http://www.comp.nus.edu.sg/~nlp/sw/sm_norm_mt.tar.gz

In [15]:
url = 'https://www.comp.nus.edu.sg/~nlp/sw/10gec_annotations.zip'
urllib.request.urlretrieve(url, './data/10gec_annotations.zip')
unpack_archive('./data/10gec_annotations.zip', './data/10gec_annotations')

#### Cornell Movie--Dialogs Corpus
url3 = http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

In [16]:
url = 'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip'
urllib.request.urlretrieve(url, './data/cornell_movie_dialogs_corpus.zip')
unpack_archive('./data/cornell_movie_dialogs_corpus.zip', './data/cornell_movie_dialogs_corpus')

#### Mispelt words
url4 = https://en.wikipedia.org/wiki/Wikipedia:Lists_of_common_misspellings/For_machines

In [102]:
url = 'https://en.wikipedia.org/wiki/Wikipedia:Lists_of_common_misspellings/For_machines'
html_text = urlopen(url).read().decode("utf-8")

soup = BeautifulSoup(html_text)
soup = soup.findAll("pre")[0].renderContents().decode('latin-1').strip()
soup = soup.split('\n')

In [106]:
reviews = [['incorrect','correct']]
for string in soup:
    words = string.split('-&gt;')
    if len(words[1].split(',')) > 1:
        for word in words[1].split(','):
            reviews.append([words[0],word])
    else:
        reviews.append([words[0],words[1]])

In [107]:
with open('./data/cornell_movie_dialogs.csv', "w") as f:
    writer = csv.writer(f)
    writer.writerows(reviews)

## Investigate Data Sets
___

### Data for Spelling Checker


In [108]:
df = pd.read_csv('./data/cornell_movie_dialogs.csv')
df.head()

,incorrect,correct
0,abandonned,abandoned
1,aberation,aberration
2,abilityes,abilities
3,abilties,abilities
4,abilty,ability


In [109]:
df.shape

(4518, 2)

### Data for Grammar Checker

In [112]:
#load data
path = './data/conll14st-test/conll14st-test-data/alt/official-2014.combined-withalt.m2'
with open(path, 'r') as f:
    content = f.read()

In [304]:
#Uncomment to print print data
#print(content)

#### Preprocess data

In [116]:
grammar_data = [i.split('\n') for i in content.split('\n\n')]

In [301]:
data = [['incorrect','correct']]
for line in grammar_data:
    SS = []
    S = line[0].split(' ')
    origin = line[0].split(' ')
    index_collection = []
    
    if len(line) == 1:
        data.append([line[0][2:],line[0][2:]])
    elif len(line) != 0:
        for i in line[1:][::-1]:
            A = i.split('|||')
            index = A[0].split(' ')[1:]

            if (index[0] == index[1]) or (index[0] == '-1'):
                continue

            elif index[0] not in index_collection:
                post = A[2].split()

                if len(S[int(index[0])+1:int(index[1])+1]) < len(A[2].split()):
                    S[int(index[0])+1:int(index[1])+1] = [A[2].split()]+[' ']*(len(S[int(index[0])+1:int(index[1])+1]) - 1)
                elif len(S[int(index[0])+1:int(index[1])+1]) > len(A[2].split()):
                    S[int(index[0])+1:int(index[1])+1] = [A[2].split()]+[' ']*(len(S[int(index[0])+1:int(index[1])+1]) - len(A[2].split()))
                else:
                    S[int(index[0])+1:int(index[1])+1] = A[2].split()

        index_collection.append(index[0])

        for s in S:
            if type(s) == list:
                for i in s:
                    SS.append(i)
            else:
                SS.append(s)
        data.append([" ".join(line[0].split(' ')[1:])," ".join(SS[1:])])
    
#print(data)

In [297]:
with open('./data/conll14st-test-data.csv', "w") as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [298]:
df = pd.read_csv('./data/conll14st-test-data.csv')
df.head()

,incorrect,correct
0,Keeping the Secret of Genetic Testing,Keeping the Secret of Genetic Testing
1,What is genetic risk ?,What is genetic risk ?
2,Genetic risk refers more to your chance of inh...,Genetic risk refers to your chance of inheri...
3,People get certain disease because of genetic ...,People get certain diseases because of genetic...
4,How much a genetic change tells us about your ...,How much a genetic change tells us about your ...


In [299]:
df.shape

(1313, 2)

In [300]:
#Examine a data point
print("Incorrect:\n{}\n\nCorrect:\n{}".format(df.iloc[11]['incorrect'],df.iloc[11]['correct']))

Incorrect:
On one hand , we do not want this potential danger causing firghtenning affects in our families ' later lives .

Correct:
On the one hand we do not want this potential danger having frightening effects on our families ' later lives .
